In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
%pylab inline
%matplotlib inline

path_dataset_folder = "../dataset/"
path_dataset_train = path_dataset_folder + "training.csv"
path_dataset_train_simplified = path_dataset_folder + "training_simplified.csv"
path_dataset_test = path_dataset_folder + "test.csv"

print("Hello")
print(os.listdir(path_dataset_folder))

#left_eye_center,
#right_eye_center,
#left_eye_inner_corner,
#left_eye_outer_corner,
#right_eye_inner_corner,
#right_eye_outer_corner,
#left_eyebrow_inner_end,
#left_eyebrow_outer_end,
#right_eyebrow_inner_end,
#right_eyebrow_outer_end,
#nose_tip,
#mouth_left_corner,
#mouth_right_corner,
#mouth_center_top_lip,
#mouth_center_bottom_lip

#train = pd.read_csv(path_dataset_train)
# 15 Keypoints
# Train has 7049 images
# Test has 1783 images -> 27124 keypoints to predict
# Images are 96x96

#simplified = train.drop(columns = ["Image"])
#simplified.to_csv(path_dataset_folder + "training_simplified.csv")

train = pd.read_csv(path_dataset_train_simplified)

#print(train.loc[0])
#for c in train.loc[0][1:]:
#    print("c: ", c)
    
train_path_format = "../images/train/{}_img.png"
test_path_format = "../images/test/{}_img.png"

def add_points(data, index):
    print("index", train.loc[index][0])
    points = train.loc[index][1:]
    for c in range(0, len(points), 2):
        x = points[c]
        y = points[c + 1]
        circle = plt.Circle((x, y), radius=1.5, fc='r')
        plt.gca().add_patch(circle)

def show_image(data, path, index):
    img = plt.imread(path.format(index))
    print(img.shape)
    plt.imshow(img,cmap='gray')
    add_points(data, index)
    
# Show the image
for i in range(0, 3):
    show_image(train, train_path_format, i)
    plt.show()

In [ ]:

path_dataset_folder = "../dataset/"
path_dataset_train_simplified = path_dataset_folder + "training_simplified.csv"

data = pd.read_csv(path_dataset_train_simplified)
column_names = [c for c in data]

boxes = []
for i in range(1, len(column_names), 2):
    x_name = column_names[i]
    y_name = column_names[i + 1]
    
    upper = (data[x_name].max(), data[y_name].max())
    lower = (data[x_name].min(), data[y_name].min())
    
    uw = upper[0] - lower[0]
    uh = upper[1] - lower[1]
    
    boxes.append((lower, upper))
    print("{} {} -> max {} min {} w: {}, h: {}".format(x_name, y_name, upper, lower, uw, uh))




In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
%pylab inline
%matplotlib inline

import common as cm

loader = cm.BatchDataLoader(train_val_split = 1.0, randomize_indices = True)
def show_face_points(img, points):
    print(img.shape)
    plt.imshow(img,cmap='gray')

    for c in range(0, len(points), 2):
        x = points[c]
        y = points[c + 1]
        circle = plt.Circle((x, y), radius=1.5, fc='r')
        plt.gca().add_patch(circle)
w = 96
hw = w // 2.0

def draw_boxes():
    idx = 0
    for b in boxes:
        lower, upper = b
        
        x_min,y_min = lower
        x_max,y_max = upper
        
        xw = x_max - x_min
        xh = y_max - y_min
        if idx == 0:
            color = 'y'
            lwidth = 4
        else:
            color = 'r'
            lwidth = 1
        plt.gca().add_patch(plt.Rectangle((x_min,y_min),xw,xh,linewidth=lwidth,edgecolor=color,facecolor='none'))
        idx += 1
n = 0
for x, y in loader.train_batch_generator(1):
    show_face_points(x[0], y[0])

    #plt.gca().add_patch(plt.Rectangle((0,0),hw,hw,linewidth=1,edgecolor='r',facecolor='none'))
    #plt.gca().add_patch(plt.Rectangle((hw,hw),hw,hw,linewidth=1,edgecolor='g',facecolor='none'))
    draw_boxes()
    plt.show()

    if n > 3:
        #pass
        break
    n += 1

#rect = patches.Rectangle((50,100),40,30,linewidth=1,edgecolor='r',facecolor='none')


In [ ]:

import simple_pipeline as lr
import common as cm

loader = cm.BatchDataLoader(train_val_split = 0.7, randomize_indices = True)
# run
estimators = lr.pipeline(loader)

In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
%pylab inline
%matplotlib inline

vx, vy = loader.get_val_data()

    

In [ ]:


def render_points(points, color='r', size = 2.5):
    #print(color + "oy: ", np.shape(points))
    for c in range(0, len(points), 2):
        x = points[c]
        y = points[c + 1]
        #print(str(x) + "," + str(y))
        circle = plt.Circle((x, y), radius=size, fc=color)
        plt.gca().add_patch(circle)
        
for i, x in enumerate(vx):
    y = vy[i]
    if np.isnan(y).any():
        continue
        
    print("validation index" + str(i))
    plt.imshow(x,cmap='gray')
    render_points(y)
    
    for est_idx in range(0, len(estimators), 2):
        x_sample = x.flatten().reshape(1,-1)
        point = []
        point.extend(estimators[est_idx].predict(x_sample))
        point.extend(estimators[est_idx + 1].predict(x_sample))
        #print("point: ", point)
        render_points(point, 'g', size=1.5)
    
    
    plt.show()
    if i > 50:
        break
    
